In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
%matplotlib notebook
import load_raw as Raw
import seaborn as sns
from scipy.special import expit as logit
from scipy.special import expit as logit
from sklearn.model_selection import train_test_split
from scipy.special import expit as sigmoid # is more stable in case of overflows
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [17]:
df_cleaned = pd.read_csv('/data/ortho/AllPatients.csv',  sep= ';')

# x is naar rechts
# y is omhoog
# z is rotatie????

In [26]:
# bereken symmetrie
df_cleaned['clavicula_x_dif'] = np.absolute(df_cleaned['clavicula_l_x'] - df_cleaned['clavicula_r_x'])
df_cleaned['clavicula_y_dif'] = np.absolute(df_cleaned['clavicula_l_y'] - df_cleaned['clavicula_r_y'])
df_cleaned['clavicula_z_dif'] = np.absolute(df_cleaned['clavicula_l_z'] - df_cleaned['clavicula_r_z'])

df_cleaned['scapula_x_dif'] = np.absolute(df_cleaned['scapula_l_x'] - df_cleaned['scapula_r_x'])
df_cleaned['scapula_y_dif'] = np.absolute(df_cleaned['scapula_l_y'] - df_cleaned['scapula_r_y'])
df_cleaned['scapula_z_dif'] = np.absolute(df_cleaned['scapula_l_z'] - df_cleaned['scapula_r_z'])

df_cleaned['humerus_x_dif'] = np.absolute(df_cleaned['humerus_l_x'] - df_cleaned['humerus_r_x'])
df_cleaned['humerus_y_dif'] = np.absolute(df_cleaned['humerus_l_y'] - df_cleaned['humerus_r_y'])
df_cleaned['humerus_z_dif'] = np.absolute(df_cleaned['humerus_l_z'] - df_cleaned['humerus_r_z'])

# hulp array, met alle parameters die voor de classifier gebruikt worden, je kan hier alles in doen wat je wilt
param = [ \
          'humerus_l_x', 'humerus_l_y', 'humerus_l_z', 'humerus_r_x', 'humerus_r_y', 'humerus_r_z', \
          'clavicula_l_x', 'clavicula_l_y', 'clavicula_l_z', 'clavicula_r_x', 'clavicula_r_y', 'clavicula_r_z', \
          'scapula_l_x', 'scapula_l_y', 'scapula_l_z', 'scapula_r_x', 'scapula_r_y', 'scapula_r_z', \
          'clavicula_x_dif','clavicula_y_dif','clavicula_z_dif', \
          'scapula_x_dif','scapula_y_dif','scapula_z_dif', \
          'humerus_x_dif', 'humerus_y_dif', 'humerus_z_dif'
         ]

df_cleaned['bias'] = 1

# split oorsprong kolom in onderdelen
x,y = df_cleaned['Oorsprong'].str.split(".").str #Oordprong word vertaald naar een string en wordt gesplits op de punt
df_cleaned['cat'],df_cleaned['pat'],df_cleaned['meting'],df_cleaned['oef'] = x.str.split("_").str #4 categorieen gemaakt obv file name
df_cleaned['cat'] = [ int(x[3:]) for x in df_cleaned['cat']] #voor elk 3+ element in de kolom wordt vertaald naar een int
df_cleaned['meting'] = [ int(x[6:]) for x in df_cleaned['meting']] 
df_cleaned['oef'] = [ int(x[3:]) for x in df_cleaned['oef']] 
df_cleaned['pat'] = [ int(x[3:]) for x in df_cleaned['pat']] 
#na deze regels te hebben uitgevoerd zijn er nieuwe categorieen met ints.

df_cleaned['pat'] = df_cleaned['cat']*1000+df_cleaned['pat'] #geef elke patient een uniek nummer

#maak boolean kolom per categorie
df_cleaned['c4'] = ['Cat4' in vincent for vincent in df_cleaned['Oorsprong']]
df_cleaned['c3'] = ['Cat3' in vincent for vincent in df_cleaned['Oorsprong']]
df_cleaned['c2'] = ['Cat2' in vincent for vincent in df_cleaned['Oorsprong']]
df_cleaned['c1'] = ['Cat1' in vincent for vincent in df_cleaned['Oorsprong']]

#hulpvariabele geeft aan welke categorie we willen onderscheiden
class_2b_found = 'c4'

Xcolumns = ['bias']
Xcolumns.extend(param)

# 
X = df_cleaned[Xcolumns]
y = df_cleaned[class_2b_found]



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [29]:
# De keuze voor een classifier is "LogisticRegression", vandaar dat lr= LogisticRegression
lr = LogisticRegression()

# Fit het model, met de train data (80 % )
lr.fit(X_train, y_train)

# predict of de test data goed past bij deze LogisticRegression
y_pred = lr.predict(X_test)

# print de accuratie score
print('Accuratie test set')
print(metrics.accuracy_score(y_test, y_pred))

y_pred = lr.predict(X_train)
print('Accuratie trainings set')
print(metrics.accuracy_score(y_train, y_pred))

Accuratie test set
0.965120009332439
Accuratie trainings set
0.9670074078394774


## Einde LogisticRegression

# Begin KKN regression

In [13]:
# try K=1 through K=25 and record testing accuracy
k_range = range(1, 10)

# We can create Python dictionary using [] or dict()
scores = []

# We use a loop through the range 1 to 26
# We append the scores in the dictionary
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    scores.append(metrics.accuracy_score(y_test, y_pred))

print(scores)

# plot the relationship between K and testing accuracy
# plt.plot(x_axis, y_axis)
plt.plot(k_range, scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Testing Accuracy')



[0.9995333780512701, 0.999154247717927, 0.9993000670769051, 0.9988626089999708, 0.9988334451281752, 0.9985418064102191, 0.9988042812563795, 0.9982501676922628, 0.9983376593076497]


Text(0,0.5,'Testing Accuracy')

In [14]:
# instantiate the model with the best known parameters
knn = KNeighborsClassifier(n_neighbors=1)

# train the model with X and y (not X_train and y_train)
knn.fit(X, y)

# make a prediction for an out-of-sample observation
knn.predict()

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """


TypeError: predict() missing 1 required positional argument: 'X'

In [ ]:

TP = 0
for i in range(x.shape[0]):
    if x[i] == y[i] == True:
        TP = TP+1

TN = 0
for i in range(x.shape[0]):
    if x[i] == y[i] == False:
        TN = TN+1

FP = 0
for i in range(x.shape[0]):
    if (x[i] == True) and (y[i] == False):
        FP = FP+1

FN = 0
for i in range(x.shape[0]):
    if (x[i] == False) and (y[i] == True):
        FN = FN+1

tab = [["pred pos", TP, FP], ["pred neg", FN, TN]]
print(pd.DataFrame(tab, columns=["", "pos", "neg"]))
print()
print("recall", TP/(TP+FN))
print("precision", TP/(TP+FP))
print("accuracy", (TP+TN)/(TP+TN+FP+FN))


In [ ]:
x = predict(X2, theta)

#True positives tellen
TP = 0
for i in range(x.shape[0]):
    if x[i] == y2[i] == True:
        TP = TP+1
#True negatives tellen
TN = 0
for i in range(x.shape[0]):
    if x[i] == y2[i] == False:
        TN = TN+1
#False positives tellen
FP = 0
for i in range(x.shape[0]):
    if (x[i] == True) and (y2[i] == False):
        FP = FP+1
#False negatives
FN = 0
for i in range(x.shape[0]):
    if (x[i] == False) and (y2[i] == True):
        FN = FN+1
        
#tabel printen
tab = [["pred pos", TP, FP], ["pred neg", FN, TN]]
print("resultaten op testset")
print(pd.DataFrame(tab, columns=["", "pos", "neg"]))
print()
print("recall", TP/(TP+FN))
print("precision", TP/(TP+FP))
print("accuracy", (TP+TN)/(TP+TN+FP+FN))


In [ ]:
# kolom met voorspelling toevoegen aan data
Xtmp = np.matrix(df_cleaned[Xcolumns])
df_cleaned['predict'] = predict(Xtmp, theta)



In [ ]:
## maak tabel (df4) met per patient het aantal sampels dat true en false gelabeld wordt.

# aantal false tellen
df2 = df_cleaned.groupby(['pat','predict']).size().to_frame('countF').reset_index()
df2 = df2.where(df2['predict'] == False).dropna()
df2 = df2.set_index('pat')

# aantal true tellen
df3 = df_cleaned.groupby(['pat','predict']).size().to_frame('countT').reset_index()
df3 = df3.where(df3['predict'] == True).dropna()
df3 = df3.set_index('pat')

# join df2 en df3
df4 = pd.concat([df2, df3], axis=1)
df4.drop(['predict','predict'], axis=1, inplace=True)
df4.fillna(0, inplace=True)

In [ ]:
# bereken percentage
df4['percentage'] = 100*df4['countT'] / (df4['countT'] + df4['countF'])
df4.sort_values(by=['percentage'],ascending=False).head(30)